In [1]:
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai.tools import tool
from IPython.display import display, Markdown
import requests

load_dotenv()

/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


True

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
synth_llm_small = LLM(
    model="anthropic/claude-3-5-haiku-20241022",
    temperature=0.2
)
synth_llm_large = LLM(
    model="anthropic/claude-3-5-sonnet-20241022",
    temperature=0.7
)

In [4]:
finhealth_sales_analyst_agent = Agent(
    config=agents_config['finhealth_sales_analyst'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=synth_llm_small,
    max_rpm=5,
    respect_context_window=True
)
chief_sales_analyst_agent = Agent(
    config=agents_config['chief_sales_analyst'],
    llm=synth_llm_large,
    max_rpm=25,
    respect_context_window=True
)

In [5]:
research_tasks = [
    Task(config=tasks_config[tc], agent=finhealth_sales_analyst_agent, async_execution=True) \
    for tc in list(tasks_config.keys()) \
    if tc.startswith('finhealth_research')
]

finhealth_report_task = Task(
   config=tasks_config['finhealth_report_task'],
    agent=chief_sales_analyst_agent,
    context=research_tasks
)

In [6]:
finhealth_crew = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[
        *research_tasks,
        finhealth_report_task
    ],
    verbose=True
)

In [ ]:
company = 'Golden 1 Credit Union'
result = finhealth_crew.kickoff(inputs={
  'Company': company
})

# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their board commitment to financial health and wellness. Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing the committment of their board to financial health and wellness. Prioritize information in the \ last 2-3 years, given that the current year is 2025. Only use information that  you found in your search.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria,  in order of decreasing importance: - Published strategic plans or annual reports mentioning financial health initiatives - Public statements from board members about financial health priorities - Dedicated financial health sections on website/marketing materials - Board meeting minutes (if public) discussing financial health programs - Financial health 

2025-02-14 14:40:39,682 - 12918534144 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 10,000 output tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}
Exception in thread Thread-9 (_execute_task_async):
Traceback (most recent call last):
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/handler.py", line 412, in completion
    response = client.post(
               ^^^^^^^^^^^^
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/l



LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

 Error during LLM call: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 10,000 output tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}


LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

 Error during LLM call: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 10,000 output tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-lim

KeyboardInterrupt: 

2025-02-14 14:40:49,041 - 12969013248 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 50,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}
Exception in thread Thread-12 (_execute_task_async):
Traceback (most recent call last):
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/handler.py", line 412, in completion
    response = client.post(
               ^^^^^^^^^^^^
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/l



# Agent: Lead Sales Analyst - Financial Health and Wellness
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Golden 1 Credit Union financial health wellness annual report board commitment 2023 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'Golden 1 Credit Union financial health wellness annual report board commitment 2023 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Newsroom | Golden 1 Credit Union', 'link': 'https://www.golden1.com/discoverg1/newsroom', 'snippet': '... committed to enhancing the financial well-being of Californians and their diverse communities. ... Annual Report 2023 · Membership · Financial Wellness Center.', 'position': 1}, {'title': 'About Us | Golden 1 Credit Union', 'link': 'https://www.golden1.com/discoverg1/about-us', 'snippet': '2023 Annual Report. Learn more about our financial and operational results and corporate giving programs. Download the Annual Report.', 'position': 2}, {'

2025-02-14 14:40:49,592 - 12952186880 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 50,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}
Exception in thread Thread-11 (_execute_task_async):
Traceback (most recent call last):
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/handler.py", line 412, in completion
    response = client.post(
               ^^^^^^^^^^^^
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/l



# Agent: Lead Sales Analyst - Financial Health and Wellness
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Golden 1 Credit Union board commitment to financial wellness annual report 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'Golden 1 Credit Union board commitment to financial wellness annual report 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Newsroom | Golden 1 Credit Union', 'link': 'https://www.golden1.com/discoverg1/newsroom', 'snippet': '... committed to enhancing the financial well-being of Californians and their diverse communities. ... Annual Report 2023 · Membership · Financial Wellness Center.', 'position': 1}, {'title': 'About Us | Golden 1 Credit Union', 'link': 'https://www.golden1.com/discoverg1/about-us', 'snippet': '2023 Annual Report. Learn more about our financial and operational results and corporate giving programs. Download the Annual Report.', 'position': 2}, {'title': '[PDF] 202

2025-02-14 14:40:49,772 - 12935360512 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 50,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}
Exception in thread Thread-10 (_execute_task_async):
Traceback (most recent call last):
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/litellm/llms/anthropic/chat/handler.py", line 412, in completion
    response = client.post(
               ^^^^^^^^^^^^
  File "/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/l



LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

 Error during LLM call: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 50,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}


# Agent: Lead Sales Analyst - Financial Health and Wellness
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Golden 1 Credit Union board commitment to financial wellness iGrad partnership 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'Golden 1 Credit Union board commitment to financial wellness iGrad partnership 2023', 'type'

In [10]:
with open('output/%s.md'%company, 'w') as file:
    file.write(str(result))

In [9]:
for n in range(len(research_tasks)):
    _task = research_tasks[n]
    _crew = Crew(
        agents=[finhealth_sales_analyst_agent],
        tasks=[_task],
        verbose=True
    )
    _result = _crew.kickoff(inputs={
        'Company': company
    })
    with open('output/%s - %d.md' % (company,n+1), 'w') as file:
        file.write(str(_result))


2025-02-14 14:22:45,135 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their board commitment to financial health and wellness. Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing the committment of their board to financial health and wellness. Prioritize information in the \ last 2-3 years, given that the current year is 2025. Only use information that  you found in your search.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria,  in order of decreasing importance: - Published strategic plans or annual reports mentioning financial health initiatives - Public statements from board members about financial health priorities - Dedicated financial health sections on website/marketing materials - Board meeting minutes (if public) discussing financial health programs - Financial health 

2025-02-14 14:23:22,061 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Board Commitment to Financial Health and Wellness Evaluation for Golden 1 Credit Union

Overall Rating: 4/5 (Strong Commitment)

Detailed Evaluation:

1. Published Strategic Plans and Initiatives:
- Launched Enrich™ Financial Wellness Platform in May 2023 in partnership with iGrad
- Platform provides personalized financial education with key features:
  * Your Money Personality™ financial behavior assessment
  * Personalized financial education courses
  * Retirement and home affordability analyzers
  * Student loan and higher education tools
- Demonstrated measurable impact from platform:
  * 35% increase in users on track with savings goals
  * 14% increase in retirement savings contributions
  * 59% increase in emergency savings
  * 28% increase in full credit card debt payment

Sources:
- https://enrich.org/golden-1-credit-union-teams-up-with-igrad-to-offer-the-enrich-personalized-financial-wellness-pro

2025-02-14 14:23:51,827 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


...



# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Golden 1 Credit Union - Financial Health and Wellness Evaluation Report

Overall Rating: 4/5 (Strong Evidence of Commitment to Financial Health and Wellness)

1. Member-Facing Financial Health Resources and Tools (Strongest Evidence)
- Launched Enrich™ Personalized Financial Wellness Program in partnership with iGrad (2023)
  - Digital platform replacing previous Learning Lab
  - Offers comprehensive financial education courses
  - Provides personalized action plans on topics including:
    * Budgeting
    * Mortgages
    * Healthcare
    * College savings
    * Student loan management
- Financial Wellness Center with free resources to:
  * Learn money management skills
  * Reduce financial stress
  * Improve credit
  * Gain financial peace of mind

Sources:
- https://enrich.org/golden-1-credit-union-teams-up-with-igrad-to-offer-the-enrich-personalized-financial-wellness-program-to-its-members/
- https://ww

2025-02-14 14:27:48,230 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
# Golden 1 Credit Union - Innovation Adoption Report

## Overall Innovation Adoption Rating: 4/5

### Detailed Evaluation of Innovation Initiatives

#### 1. Implementation of Emerging Fintech Solutions
- **NCR Voyix Digital Banking Partnership (June 2024)**
  - Selected NCR Voyix Digital-First Banking solution to drive digital transformation
  - Demonstrates commitment to cutting-edge digital banking technologies
  - Sources: 
    - NCR Investor Relations
    - Yahoo Finance
    - Candescent Insights

- **Prizeout Partners CUSO Collaboration**
  - Joined Prizeout Partners, a Credit Union Service Organization (CUSO)
  - Implemented innovative technology for member savings opportunities
  - Expanded digital revenue streams
  - Sources:
    - CU Times
    - Globe Newswire
    - CB Insights

#### 2. Beta Testing Partnerships with Technology Providers
- Actively engaged with NCR Voyix for digital banking transfo

2025-02-14 14:28:18,778 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Industry Leadership Report for Golden 1 Credit Union (2024)

OVERALL RATING: 4/5 ⭐⭐⭐⭐

Detailed Evaluation of Industry Leadership:

1. Industry Awards and Recognition
- Received 7 prestigious national recognitions in 2024
- Key Awards:
  a) American Banker - Best Credit Unions to Work For (4th consecutive year)
  b) Forbes - America's Best-In-State Employers 2024 (5th time)
  c) Forbes - America's Best Employers for Women 2024
  d) Newsweek - America's Greatest Workplaces for Women 2024
  e) Newsweek - America's Greatest Workplaces 2024
  f) Forbes - America's Best-In-State Credit Unions 2024
  g) TIME Magazine - America's Best Mid-Size Companies 2024

Sources:
- [Golden 1 Press Release: National Accolades 2024](https://www.golden1.com/discoverg1/press-releases/2024/Golden%201%20Credit%20Union%20Honored%20with%20Multiple%20National%20Accolades)

2. Speaking Slots and Conference Participation
- Limited publi